# Visium pipeline runthrough
------
In this notebook, we'll go through how to utilize Giotto for visium data. While there are some notebooks that will use visium data in this tutorial, the majority use the seqFISH dataset. You can use this notebook or the script ```vis_obj.R``` as reference.

First, we'll install Giotto:

In [ ]:
remotes::install_github("RubD/Giotto@cless")

And load Giotto:

In [ ]:
library(Giotto)

Now we can define our expression, location, and python paths:

In [ ]:
# expresstion path
expr_path = system.file("extdata", "visium_DG_expr.txt", package = 'Giotto')
# spatial locations path
loc_path = system.file("extdata", "visium_DG_locs.txt", package = 'Giotto')
# python path
my_instructions = createGiottoInstructions(python_path = "/srv/conda/envs/notebook/bin/python")

And finally, we can create our Giotto object:

In [ ]:
my_visium_object <- createGiottoObject(raw_exprs = expr_path,
                                       spatial_locs = loc_path,
                                       instructions = my_instructions)

In [ ]:
# read image
png_path = system.file("extdata", "deg_image.png", package = 'Giotto')
mg_img = magick::image_read(png_path)

In [ ]:
# test and modify image alignment
mypl = spatPlot(my_visium_object, return_plot = T, point_alpha = 0.8)
orig_png = createGiottoImage(gobject = my_visium_object, mg_object = mg_img, name = 'image',
                             xmax_adj = 450, xmin_adj = 550,
                             ymax_adj = 200, ymin_adj = 200)
mypl_image = addGiottoImageToSpatPlot(mypl, orig_png)
mypl_image


In [ ]:
# add image to Giotto object

image_list = list(orig_png)
my_visium_object = addGiottoImage(gobject = my_visium_object,
                             images = image_list)
showGiottoImageNames(my_visium_object)

In [ ]:
# explore gene and cell distribution
filterDistributions(my_visium_object, detection = 'genes')
filterDistributions(my_visium_object, detection = 'cells')
filterCombinations(my_visium_object,
                   expression_thresholds = c(1),
                   gene_det_in_min_cells = c(20, 20, 50, 50),
                   min_det_genes_per_cell = c(100, 200, 100, 200))

# filter and normalize
my_visium_object <- filterGiotto(gobject = my_visium_object,
                            expression_threshold = 1,
                            gene_det_in_min_cells = 50,
                            min_det_genes_per_cell = 100,
                            expression_values = c('raw'),
                            verbose = T)
my_visium_object <- normalizeGiotto(gobject = my_visium_object, scalefactor = 6000, verbose = T)
my_visium_object <- addStatistics(gobject = my_visium_object)

In [ ]:
my_visium_object <- calculateHVG(gobject = my_visium_object)

my_visium_object <- runPCA(gobject = my_visium_object)
screePlot(my_visium_object, ncp = 30)[0]
plotPCA(gobject = my_visium_object)[0]

my_visium_object <- runUMAP(my_visium_object, dimensions_to_use = 1:10)
plotUMAP(gobject = my_visium_object)[0]
my_visium_object <- runtSNE(my_visium_object, dimensions_to_use = 1:10)
plotTSNE(gobject = my_visium_object)[0]

In [ ]:
my_visium_object <- createNearestNetwork(gobject = my_visium_object, dimensions_to_use = 1:10, k = 20)
my_visium_object <- doLeidenCluster(gobject = my_visium_object, resolution = 0.4, n_iterations = 1000)

pDataDT(my_visium_object)

# visualize UMAP cluster results
plotUMAP(gobject = my_visium_object, cell_color = 'leiden_clus', show_NN_network = T, point_size = 2.5)

# visualize UMAP and spatial results
spatDimPlot(gobject = my_visium_object, cell_color = 'leiden_clus', spat_point_shape = 'voronoi')

# heatmap and dendrogram
showClusterHeatmap(gobject = my_visium_object, cluster_column = 'leiden_clus')
showClusterDendrogram(my_visium_object, h = 0.5, rotate = T, cluster_column = 'leiden_clus')

In [ ]:
scran_markers = findMarkers_one_vs_all(gobject = my_visium_object,
                                       method = 'scran',
                                       expression_values = 'normalized',
                                       cluster_column = 'leiden_clus')
# # violinplot
# topgenes_scran = scran_markers[, head(.SD, 2), by = 'cluster']$genes
# violinPlot(my_visium_object, genes = topgenes_scran, cluster_column = 'leiden_clus',
#            strip_text = 10, strip_position = 'right')

# # metadata heatmap
# topgenes_scran = scran_markers[, head(.SD, 6), by = 'cluster']$genes
# plotMetaDataHeatmap(my_visium_object, selected_genes = topgenes_scran,
#                     metadata_cols = c('leiden_clus'))

In [ ]:

cluster_cell_types = c('Gfap_cells', 'Tbr1_cells', 'Tcf7l2_cells', 'Wfs1_cells', 'Nptxr_cells')
names(cluster_cell_types) = 1:5
my_visium_object = annotateGiotto(gobject = my_visium_object, annotation_vector = cluster_cell_types,
                             cluster_column = 'leiden_clus', name = 'cluster_cell_types')

# check new cell metadata
pDataDT(my_visium_object)

# visualize annotations
spatDimPlot(gobject = my_visium_object, cell_color = 'cluster_cell_types', spat_point_size = 3, dim_point_size = 3)

In [ ]:
## cell type signatures ##
## combination of all marker genes identified in Zeisel et al
sign_matrix_path = system.file("extdata", "sig_matrix.txt", package = 'Giotto')
brain_sc_markers = data.table::fread(sign_matrix_path) # file don't exist in data folder
sig_matrix = as.matrix(brain_sc_markers[,-1]); rownames(sig_matrix) = brain_sc_markers$Event

## enrichment tests
my_visium_object = runSpatialEnrich(my_visium_object, 
                               sign_matrix = sig_matrix, 
                               enrich_method = 'PAGE') #default = 'PAGE'

## heatmap of enrichment versus annotation (e.g. clustering result)
cell_types = colnames(sig_matrix)
plotMetaDataCellsHeatmap(gobject = my_visium_object,
                         metadata_cols = 'leiden_clus',
                         value_cols = cell_types,
                         spat_enr_names = 'PAGE',
                         x_text_size = 8, y_text_size = 8)


enrichment_results = my_visium_object@spatial_enrichment$PAGE
enrich_cell_types = colnames(enrichment_results)
enrich_cell_types = enrich_cell_types[enrich_cell_types != 'cell_ID']

## spatplot
spatCellPlot(gobject = my_visium_object, spat_enr_names = 'PAGE',
             cell_annotation_values = enrich_cell_types,
             cow_n_col = 3,coord_fix_ratio = NULL, point_size = 1)

In [ ]:
my_visium_object <- createSpatialGrid(gobject = my_visium_object,
                                 sdimx_stepsize = 300,
                                 sdimy_stepsize = 300,
                                 minimum_padding = 50)
showGrids(my_visium_object)
spatPlot(gobject = my_visium_object, show_grid = T, point_size = 1.5)

# extract grid and associated metadata spots
annotated_grid = annotateSpatialGrid(my_visium_object)
annotated_grid_metadata = annotateSpatialGrid(my_visium_object,
                                              cluster_columns = c('leiden_clus', 'cell_types', 'nr_genes'))

In [ ]:

plotStatDelaunayNetwork(gobject = my_visium_object, maximum_distance = 300)
my_visium_object = createSpatialNetwork(gobject = my_visium_object, minimum_k = 2, maximum_distance_delaunay = 400)
my_visium_object = createSpatialNetwork(gobject = my_visium_object, minimum_k = 2, method = 'kNN', k = 10)
showNetworks(my_visium_object)

# visualize the two different spatial networks  
spatPlot(gobject = my_visium_object, show_network = T,
         network_color = 'blue', spatial_network_name = 'Delaunay_network',
         point_size = 2.5, cell_color = 'leiden_clus')

spatPlot(gobject = my_visium_object, show_network = T,
         network_color = 'blue', spatial_network_name = 'kNN_network',
         point_size = 2.5, cell_color = 'leiden_clus')

In [ ]:
km_spatialgenes = binSpect(my_visium_object)
spatGenePlot(my_visium_object, expression_values = 'scaled', 
             genes = km_spatialgenes[1:4]$genes,
             point_shape = 'border', point_border_stroke = 0.1,
             show_network = F, network_color = 'lightgrey', point_size = 2.5,
             cow_n_col = 2)[0]

rank_spatialgenes = binSpect(my_visium_object, bin_method = 'rank')
spatGenePlot(my_visium_object, expression_values = 'scaled', 
             genes = rank_spatialgenes[1:4]$genes,
             point_shape = 'border', point_border_stroke = 0.1,
             show_network = F, network_color = 'lightgrey', point_size = 2.5,
             cow_n_col = 2)[0]

In [ ]:
silh_spatialgenes = silhouetteRank(gobject = my_visium_object) # TODO: suppress print output
spatGenePlot(my_visium_object, expression_values = 'scaled', 
             genes = silh_spatialgenes[1:4]$genes,
             point_shape = 'border', point_border_stroke = 0.1,
             show_network = F, network_color = 'lightgrey', point_size = 2.5,
             cow_n_col = 2)

In [ ]:

# 1. calculate spatial correlation scores 
ext_spatial_genes = km_spatialgenes[1:100]$genes
spat_cor_netw_DT = detectSpatialCorGenes(my_visium_object,
                                         method = 'network', spatial_network_name = 'Delaunay_network',
                                         subset_genes = ext_spatial_genes)

# 2. cluster correlation scores
spat_cor_netw_DT = clusterSpatialCorGenes(spat_cor_netw_DT, name = 'spat_netw_clus', k = 8)
heatmSpatialCorGenes(my_visium_object, spatCorObject = spat_cor_netw_DT, use_clus_name = 'spat_netw_clus')


netw_ranks = rankSpatialCorGroups(my_visium_object, spatCorObject = spat_cor_netw_DT, use_clus_name = 'spat_netw_clus')
top_netw_spat_cluster = showSpatialCorGenes(spat_cor_netw_DT, use_clus_name = 'spat_netw_clus',
                                            selected_clusters = 6, show_top_genes = 1)

cluster_genes_DT = showSpatialCorGenes(spat_cor_netw_DT, use_clus_name = 'spat_netw_clus', show_top_genes = 1)
cluster_genes = cluster_genes_DT$clus; names(cluster_genes) = cluster_genes_DT$gene_ID

my_visium_object = createMetagenes(my_visium_object, gene_clusters = cluster_genes, name = 'cluster_metagene')
spatCellPlot(my_visium_object,
             spat_enr_names = 'cluster_metagene',
             cell_annotation_values = netw_ranks$clusters,
             point_size = 1.5, cow_n_col = 3)